In [1]:
import sqlite3


def retrieve_repo_test_counts(database_path, when_location):
    with sqlite3.connect(database_path) as connection:
        return connection.cursor().execute(
            'SELECT repo_name, count(*) AS test_count '
            'FROM test_relations '
            'WHERE when_location=\'GIVEN\' '
            'GROUP BY repo_name '
            'ORDER BY test_count DESC'
        ).fetchall()


def split_repos(repo_test_counts, split_distribution):
    splits = ({'repos': [], 'size': 0}, {'repos': [], 'size': 0}, {'repos': [], 'size': 0})
    distributed_count = 0

    for repo_name, test_count in repo_test_counts:
        relevant_split_index = next(
            (
                split_index for split_index, split in enumerate(splits)
                if distributed_count != 0 and split['size']/distributed_count < split_distribution[split_index]
            ),
            0
        )
        splits[relevant_split_index]['repos'].append(repo_name)
        splits[relevant_split_index]['size'] += test_count
        distributed_count += test_count

    return splits


def create_data_split(database_path, when_location, split):
    assert len(split) == 3
    assert sum(split) == 1

    repo_test_counts = retrieve_repo_test_counts(database_path, when_location)
    return split_repos(repo_test_counts, split)

In [4]:
split = create_data_split('/home/user/jdepoix/data/test_relations_index.sqlite', 'WHEN', (.8,.1,.1))

OperationalError: unable to open database file